In [4]:
import pandas as pd

In [5]:
df = pd.read_csv('/kaggle/input/cmi-detect-behavior-with-sensor-data/train.csv')
print(df.head())

              row_id sequence_type sequence_id  sequence_counter      subject  \
0  SEQ_000007_000000        Target  SEQ_000007                 0  SUBJ_059520   
1  SEQ_000007_000001        Target  SEQ_000007                 1  SUBJ_059520   
2  SEQ_000007_000002        Target  SEQ_000007                 2  SUBJ_059520   
3  SEQ_000007_000003        Target  SEQ_000007                 3  SUBJ_059520   
4  SEQ_000007_000004        Target  SEQ_000007                 4  SUBJ_059520   

                       orientation                                   behavior  \
0  Seated Lean Non Dom - FACE DOWN  Relaxes and moves hand to target location   
1  Seated Lean Non Dom - FACE DOWN  Relaxes and moves hand to target location   
2  Seated Lean Non Dom - FACE DOWN  Relaxes and moves hand to target location   
3  Seated Lean Non Dom - FACE DOWN  Relaxes and moves hand to target location   
4  Seated Lean Non Dom - FACE DOWN  Relaxes and moves hand to target location   

        phase             

In [6]:
df = df.drop('row_id', axis=1)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 574945 entries, 0 to 574944
Columns: 340 entries, sequence_type to tof_5_v63
dtypes: float64(332), int64(1), object(7)
memory usage: 1.5+ GB


In [8]:
df.describe()

,sequence_counter,acc_x,acc_y,acc_z,rot_w,rot_x,rot_y,rot_z,thm_1,thm_2,...,tof_5_v54,tof_5_v55,tof_5_v56,tof_5_v57,tof_5_v58,tof_5_v59,tof_5_v60,tof_5_v61,tof_5_v62,tof_5_v63
count,574945.000000,574945.000000,574945.000000,574945.000000,571253.000000,571253.000000,571253.000000,571253.000000,567958.000000,567307.000000,...,544803.000000,544803.000000,544803.000000,544803.000000,544803.000000,544803.000000,544803.000000,544803.000000,544803.000000,544803.000000
mean,43.645234,1.639980,1.790704,-0.459811,0.360375,-0.119916,-0.059953,-0.188298,27.076450,27.133483,...,29.395651,26.030826,45.342583,43.074842,40.045908,37.631707,34.977928,31.934330,29.024752,27.320358
std,45.717673,5.781259,5.003945,6.096490,0.225739,0.465520,0.543028,0.504137,3.231948,2.941438,...,58.093844,54.215523,68.466064,68.017631,66.941587,65.288710,63.201604,60.440645,57.218513,55.407192
min,0.000000,-34.585938,-24.402344,-42.855469,0.000000,-0.999146,-0.999695,-0.998169,-0.370413,21.958820,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,17.000000,-2.964844,-2.121094,-5.417969,0.180237,-0.456299,-0.511536,-0.627686,24.753527,24.543737,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
50%,35.000000,2.972656,0.695312,-1.562500,0.340332,-0.186890,-0.112610,-0.263916,26.982324,26.354338,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
75%,54.000000,6.343750,6.816406,5.164062,0.503479,0.204590,0.440063,0.251099,29.425036,29.620147,...,34.000000,24.000000,81.000000,76.000000,67.000000,59.000000,51.000000,42.000000,35.000000,31.000000
max,699.000000,46.328125,27.183594,30.078125,0.999390,0.999817,0.999451,0.999878,38.457664,37.578339,...,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000


In [9]:
df.isna().sum().head(25)

sequence_type           0
sequence_id             0
sequence_counter        0
subject                 0
orientation             0
behavior                0
phase                   0
gesture                 0
acc_x                   0
acc_y                   0
acc_z                   0
rot_w                3692
rot_x                3692
rot_y                3692
rot_z                3692
thm_1                6987
thm_2                7638
thm_3                6472
thm_4                6224
thm_5               33286
tof_1_v0             6224
tof_1_v1             6224
tof_1_v2             6224
tof_1_v3             6224
tof_1_v4             6224
dtype: int64

1. Not every column has missing values.
2. Columns with rotation, themometer and tof has missing values
3. Since they are in a series, they can be filled using linear interpolation.

In [10]:
df.nunique()

sequence_type          2
sequence_id         8151
sequence_counter     700
subject               81
orientation            4
                    ... 
tof_5_v59            251
tof_5_v60            251
tof_5_v61            251
tof_5_v62            251
tof_5_v63            251
Length: 340, dtype: int64

# Encode Categorical Data

In [11]:
obj_columns = [col for col in df.columns if df[col].dtype == 'object']
print(obj_columns)

['sequence_type', 'sequence_id', 'subject', 'orientation', 'behavior', 'phase', 'gesture']


In [12]:
df[obj_columns].head()

,sequence_type,sequence_id,subject,orientation,behavior,phase,gesture
0,Target,SEQ_000007,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin
1,Target,SEQ_000007,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin
2,Target,SEQ_000007,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin
3,Target,SEQ_000007,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin
4,Target,SEQ_000007,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin


In [13]:
df[obj_columns].nunique()

sequence_type       2
sequence_id      8151
subject            81
orientation         4
behavior            4
phase               2
gesture            18
dtype: int64

In [14]:
df[obj_columns].isna().sum()

sequence_type    0
sequence_id      0
subject          0
orientation      0
behavior         0
phase            0
gesture          0
dtype: int64

$\text{\large There are seven categorical columns}$ \
$\text{\large None of them have missing values}$ \
$\text{\large One-hot Encoding can create too many columns}$ \
$\text{\large Label Encoding is a good option}$ 

# Handline Missing Values

In [15]:
missing_df = df.isna().sum()
missing_df = missing_df / len(df)
print(missing_df)

sequence_type       0.000000
sequence_id         0.000000
sequence_counter    0.000000
subject             0.000000
orientation         0.000000
                      ...   
tof_5_v59           0.052426
tof_5_v60           0.052426
tof_5_v61           0.052426
tof_5_v62           0.052426
tof_5_v63           0.052426
Length: 340, dtype: float64


In [16]:
print(missing_df.values > 0.05)

[False False False False False False False False False False False False
 False False False False False False False  True False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False Fa

In [17]:
print(missing_df.values > 0.06)

[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False Fa

$\text{\large Less than 6 percent data is missing}$

In [18]:
df = df.interpolate(method='linear')

/tmp/ipykernel_35/2145716054.py:1: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df = df.interpolate(method='linear')


In [19]:
print(df.isna().sum().sum())

0


# Pipeline

In [20]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder

categorical_pipeline = Pipeline(steps=[
    ('encoder', LabelEncoder())
])

In [21]:
demo = pd.read_csv("/kaggle/input/cmi-detect-behavior-with-sensor-data/train_demographics.csv")
print(len(demo))

81
